In [1]:
# Cell 1: Imports
import os
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, random_split

from gan_utils.io import H5ImageDataset
from gan.model import Generator, Discriminator, Encoder, DiscriminatorFeatures
from gan.trainer import Trainer

import matplotlib.pyplot as plt




In [2]:
# Cell 2: Utilities

def set_seed(seed: int = 42):
    torch.manual_seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


def ensure_dir(path: str):
    if path and not os.path.exists(path):
        os.makedirs(path, exist_ok=True)


def make_dataloaders(df, h5_path, batch_size=32, workers=16, val_split=0.1):
    """
    df: pandas DataFrame with file_name column that matches keys in HDF5
    h5_path: path to HDF5 file containing images
    returns: train_loader, val_loader
    """
    dataset = H5ImageDataset(df, h5_path, transforms=None)
    n_total = len(dataset)
    n_val = int(val_split * n_total)
    n_train = n_total - n_val

    g = torch.Generator().manual_seed(123)
    train_ds, val_ds = random_split(dataset, [n_train, n_val], generator=g)

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        shuffle=True,
        num_workers=workers,
        pin_memory=True,
    )
    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        shuffle=False,
        num_workers=workers,
        pin_memory=True,
    )
    return train_loader, val_loader


def build_models(n_z=256, image_size=512, in_channels=3):
    """
    Construct Generator, Discriminator, Encoder, and DiscriminatorFeatures.
    """
    gen = Generator(
        n_z=n_z, input_filt=512, final_size=image_size, out_channels=in_channels, norm=False, pool=False
    )
    disc = Discriminator(in_channels=in_channels, n_layers=6, input_size=image_size, norm=False, pool=False,
    )
#     enc = Encoder(
#         in_channels=in_channels,
#         n_z=n_z,
#         n_layers=5,
#         input_filt=64,
#     )
#     disc_feat = DiscriminatorFeatures(
#         in_channels=in_channels,
#         n_layers=6,
#         input_filt=64,
#     )
    return gen, disc


def train_gan(gen, disc, train_loader, val_loader, outdir, epochs=50, d_lr=1e-4, g_lr=1e-4, save_freq=10, lr_decay=None, decay_freq=5, device='cuda'): 
    """ Train WGAN-GP using Trainer and save checkpoints + loss metrics. """ 
    ensure_dir(outdir) 
    gen = gen.to(device) 
    disc = disc.to(device) 
    trainer = Trainer(generator=gen, discriminator=disc, savefolder=outdir, device=device) 
    G_loss_ep, D_loss_ep = trainer.train( train_data=train_loader, val_data=val_loader, epochs=epochs, dsc_learning_rate=d_lr, gen_learning_rate=g_lr, save_freq=save_freq, lr_decay=lr_decay, decay_freq=decay_freq ) 
    return G_loss_ep, D_loss_ep




In [3]:
# Cell 3: Config

h5_path = "anomaly.h5"
parquet_path = "mask_labels_anomaly.gzip"
outdir = "./outputs"
gan_ckpt_dir = os.path.join(outdir, "gan_checkpoints")

batch_size = 32
workers = 32
val_split = 0.1
image_size = 512
n_z = 256
gan_epochs = 50
g_lr = 1e-4
d_lr = 1e-4
device = "cuda" if torch.cuda.is_available() else "cpu"
seed = 42

set_seed(seed)
ensure_dir(outdir)
ensure_dir(gan_ckpt_dir)




In [4]:
# Cell 4: Load metadata + dataloaders

df = pd.read_parquet(parquet_path)
train_loader, val_loader = make_dataloaders(
    df,
    h5_path,
    batch_size=batch_size,
    workers=workers,
    val_split=val_split,
)




/users/5/raj00075/.local/lib/python3.11/site-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 16, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [5]:
# Cell 5: Build models (we only use gen, disc here)

gen, disc = build_models(
    n_z=n_z,
    image_size=image_size,
    in_channels=3,
)

gen = gen.to(device)
disc = disc.to(device)


# Quick sanity check on shapes
imgs = next(iter(train_loader))[0]
print("Real batch shape:", imgs.shape)

z = torch.randn(4, gen.n_z).to(device)
fake = gen(z)
print("Fake batch shape:", fake.shape)




Real batch shape: torch.Size([3, 512, 512])
Fake batch shape: torch.Size([4, 3, 512, 512])


In [ ]:
# Cell 6: Train GAN

print("Training GAN (WGAN-GP)...")
G_loss_ep, D_loss_ep = train_gan(gen, disc, train_loader, val_loader, gan_ckpt_dir, epochs=gan_epochs, d_lr=d_lr, g_lr=g_lr, save_freq=10, lr_decay=None, decay_freq=5, device=device) 





/users/5/raj00075/Desktop/Anomaly_detection/KM_VERITAS_GAN/gan/trainer.py:40: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler_g = GradScaler()
/users/5/raj00075/Desktop/Anomaly_detection/KM_VERITAS_GAN/gan/trainer.py:41: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler_d = GradScaler()


Training GAN (WGAN-GP)...
Epoch 1 -- lr: 1.000e-04, 1.000e-04
-------------------------------------------------------


Training:   0%|          | 0/2813 [00:00<?, ?it/s]/users/5/raj00075/Desktop/Anomaly_detection/KM_VERITAS_GAN/gan/trainer.py:62: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
/users/5/raj00075/Desktop/Anomaly_detection/KM_VERITAS_GAN/gan/trainer.py:76: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
Validation: 100%|██████████| 313/313 [00:43<00:00,  7.14it/s, gen: 8.87e+01 disc: -4.28e+00 w_dist: -5.01e+00 grad_penalty: 7.24e-01]


Epoch 2 -- lr: 1.000e-04, 1.000e-04
-------------------------------------------------------


Validation: 100%|██████████| 313/313 [00:29<00:00, 10.53it/s, gen: 2.06e+02 disc: -3.89e+01 w_dist: -4.03e+01 grad_penalty: 1.45e+00]


...

Validation: 100%|██████████| 313/313 [00:31<00:00,  9.90it/s, gen: -2.04e+03 disc: -5.70e+00 w_dist: -5.77e+00 grad_penalty: 6.50e-02]


Epoch 34 -- lr: 1.000e-04, 1.000e-04
-------------------------------------------------------


Validation: 100%|██████████| 313/313 [00:31<00:00,  9.93it/s, gen: -3.25e+03 disc: -4.83e+01 w_dist: -6.30e+01 grad_penalty: 1.46e+01]


Epoch 35 -- lr: 1.000e-04, 1.000e-04
-------------------------------------------------------


Training:   2%|▏         | 45/2813 [00:14<09:50,  4.69it/s, gen: -4.71e+02 disc: -4.87e-01 w_dist: -2.34e+00 grad_penalty: 1.86e+00] 

In [ ]:
# Cell 7: Plot GAN losses

plt.figure()
plt.plot(G_loss_ep, label="Gen loss", color="red")
plt.plot(D_loss_ep, label="Disc loss", color="blue")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(outdir, "gan_losses.png"))
plt.close()




In [ ]:
# Cell 8: Save final generator and discriminator

torch.save(gen.state_dict(), os.path.join(gan_ckpt_dir, "generator_final.pth"))
torch.save(disc.state_dict(), os.path.join(gan_ckpt_dir, "discriminator_final.pth"))

print("Done training GAN. Checkpoints in:", gan_ckpt_dir)
